# Slackformer: Simple Inference

In [2]:
import weave
from weave.ecosystem import keras as weave_keras

In [3]:
from tensorflow import keras
model = keras.models.load_model("slackformer_64_64_E5")
    #"slackformer_4_heads_32_64_E5") # <-best one  31% val acc, doesn't load because architecture changed
    #"slackformer_64_64_E5")  < actually works

slackformer = weave.save(model)

2022-08-10 19:43:31.662155: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Assets written to: /tmp/local-artifacts/KerasModel/working-V3LPWDED6L09/_obj/assets


INFO:tensorflow:Assets written to: /tmp/local-artifacts/KerasModel/working-V3LPWDED6L09/_obj/assets


## Show top predicted user id

In [4]:
weave.use(weave_keras.call_string_to_argmax(slackformer, "one does not simply save any model"))

6

## Show top predicted users

In [5]:
@weave.op(render_info={"type": "function"},
          input_type={"user_id" : weave.types.Number()},
                      output_type=weave.types.String())
def user_by_id(user_id) -> str:
    import pickle
    names = pickle.load(open("user_map_215_over_50", 'rb'))
    try:
        username = names[user_id]
        return username
    except:
        return ""

### Run this cell twice to see stuff :)

In [7]:
# try to make a table of sample inputs
prompt_list = ["I love weave", 
               "I love coffee", 
               "can we get some metrics on this", 
               "I'm curious if someone has thought about trying this for tables"
              ]
prompt_d = [{"text" : t} for t in prompt_list]
weave.panels.Table(prompt_d,
                  columns = [
                      lambda row: row["text"],
                      lambda row: weave_keras.call_string_to_argmax(slackformer, row["text"]),
                      lambda row: user_by_id(weave_keras.call_string_to_argmax(slackformer, row["text"]))
                  ]
)
                  

## Show multiple users + confidence scores
Also need to run the panel cell twice sometimes

In [8]:
import numpy as np
import pickle

@weave.op(render_info={"type": "function"},
          input_type={"conf_scores" : weave.types.List(weave.types.Number())},
                      output_type=weave.types.List(weave.types.TypedDict(
                              {"user" : weave.types.String(),
                               "score" : weave.types.Number()})))
def show_top_10_scores(conf_scores):
    conf_scores = np.asarray(conf_scores)
    # load mapping of ids to user names
    user_id_map = pickle.load(open("user_map_215_over_50", 'rb'))
    # sort scores
    scores_with_ids = [[s, i] for i, s in enumerate(conf_scores)]
    sorted_scores = sorted(scores_with_ids, key=lambda s: s[0], reverse=True)
    rows = []
    for s in sorted_scores[:10]:
        try:
            # nicely format top 10 scores
            rows.append({
                "user" : user_id_map[s[1]], "score" : float(s[0])})
        except:
            continue
    return rows

In [10]:
prompt_list = ["I love weave", 
               "I love coffee", 
               "can we get some metrics on this", 
               "I'm curious if someone has thought about trying this for tables"
              ]
prompt_d = [{"text" : t} for t in prompt_list]
weave.panels.Table(prompt_d,
                  columns = [
                      lambda row: row["text"],
                      lambda row: show_top_10_scores(
                                    weave_keras.call_string_to_confidence_scores(slackformer, 
                                                                                 row["text"])) 
                                                
                  ]
)
                  